In [ ]:
# Basic packages
import os
import torch
import numpy as np
import platform
# Import options
from config import get_options
from configx_cfg import get_cfg_options
# Import agent class
from designx import DesignX, ConfigX
# import Modular-EC modules
from components.operators import *
from env.optimizer_env import Optimizer

from utils.utils import set_seed
from utils.make_dataset import get_test_problems
from utils.plots import display_optimizer
set_seed(2025)

Firstly let's load the models of Agent-1 and Agent-2:

In [ ]:
# Load Agent-1
load_path_1 = "models/Agent-1.pt"
agent_1 = DesignX()
agent_1.load(load_path_1)
agent_1.eval()

# Load Agent-2
load_path_2 = "models/epoch-12.pt"
agent_2 = ConfigX()
agent_2.load(load_path_2)
agent_2.eval()

Now we get the 20 testing problems presented in the paper:

In [ ]:
test_problems = get_test_problems()

Then we calculate the ELA featuers of these problems:

In [ ]:
ela_feature = agent_1.get_ELA_features(test_problems)

In [ ]:
_, _, modules, _, _ = agent_1.forward(ela_feature, rollout=True)

Let's display the structure of the generated optimizer for the first problem:

In [ ]:
display_optimizer(modules[0])

Process the modules and construct the RL environment (Optimizer)

In [ ]:
envs = []
for problem, module in zip(test_problems, modules):
    envs.append(Optimizer(problem, copy.deepcopy(module)))

Now we can optimize the problems with the generated optimizers controlled by Agent-2:

In [ ]:
_, cost, _ = agent_2.rollout(envs, repeat=19)
print(np.mean(cost, -1))